In [159]:
import requests   
import json
import pprint
import numpy as np
import pandas as pd
from functools import reduce

### 1.a Static Data

### 1.b Dynamic data

In [220]:
class WorldBankDataFetcher:
    """
    """
    def __init__(self, indicators):
        self.indicators = indicators
        self.jsons = self.dataframes = []
        self.merged_data = None        

    def fetch_json_by(indicator):
        print('Fetching:' + indicator)
        url = "http://api.worldbank.org/v2/countries/all/indicators/%s&format=json&per_page=9000" % (indicator + '?date=2006:2018')
        r = requests.get(url)    
        return json.loads(r.content)

    def json_to_df(j):
        indicator_name = j[1][0]['indicator']['value']
        df = pd.DataFrame.from_dict(j[1])
        df['country'] = df[['country']].applymap(lambda x : x['value'])
        df = df[['country', 'countryiso3code', 'date', 'value']]
        df.columns = ['country', 'code', 'date', indicator_name]
        return df
    
    def fetch(self):
        print('Note: this operation make take a while depending on the network status...\n...')      
        
        for i in indicators:
            self.jsons.append(fetch_json_by(i))
            progress = str(indicators.index(i)) + '/' + str(len(indicators))             
            print(' '.join([progress, 'Indicator', i]))
        
        print('Creating data frames...')
        self.dataframes = [json_to_df(j) for j in self.jsons]
        
        print('Merging data frames...')
        self.merged_data = reduce(lambda x, y: x.merge(y, how='outer', on=['country', 'code', 'date']), self.dataframes)        
        
        print('All done!')

In [208]:
indicators = [
    'NY.GDP.PCAP.CD',    # GDP per capita (current US$)    
    'AG.LND.AGRI.ZS',    # Agricultural land (% of land area)
    'EG.ELC.ACCS.ZS',    # Access to electricity (% of population)    
    'EN.ATM.CO2E.PC',    # CO2 emissions (metric tons per capita)
    'MS.MIL.XPND.GD.ZS', # Military expenditure (% of GDP)
    'SL.UEM.TOTL.ZS',    # Unemployment, total (% of total labor force) (modeled ILO estimate)
    'BM.GSR.ROYL.CD',    # Charges for the use of intellectual property, payments (BoP, current US$)
    'SP.POP.TOTL',       # Population, total
    'IS.AIR.DPRT',       # Air transport, registered carrier departures worldwide
    'IC.TAX.TOTL.CP.ZS'  # Total tax rate (% of commercial profits)
]

In [221]:
fetcher = WorldBankDataFetcher(indicators)
fetcher.fetch()

Note: this operation make take a while depending on the network status...
...
0/10 Indicator NY.GDP.PCAP.CD
1/10 Indicator AG.LND.AGRI.ZS
2/10 Indicator EG.ELC.ACCS.ZS
3/10 Indicator EN.ATM.CO2E.PC
4/10 Indicator MS.MIL.XPND.GD.ZS
5/10 Indicator SL.UEM.TOTL.ZS
6/10 Indicator BM.GSR.ROYL.CD
7/10 Indicator SP.POP.TOTL
8/10 Indicator IS.AIR.DPRT
9/10 Indicator IC.TAX.TOTL.CP.ZS
Creating data frames...
Merging data frames...
All done!


In [222]:
#fetcher.merged_data#[fetcher.merged_data.date == '2018']
d = fetcher.merged_data
d[d.code!='']

,country,code,date,GDP per capita (current US$),Agricultural land (% of land area),Access to electricity (% of population),CO2 emissions (metric tons per capita),Military expenditure (% of GDP),"Unemployment, total (% of total labor force) (modeled ILO estimate)","Charges for the use of intellectual property, payments (BoP, current US$)","Population, total","Air transport, registered carrier departures worldwide",Total tax rate (% of commercial profits)
611,Afghanistan,AFG,2018,NaN,NaN,NaN,NaN,NaN,8.808000,NaN,NaN,NaN,71.4
612,Afghanistan,AFG,2017,550.068459,NaN,NaN,NaN,0.906857,8.837000,1.806604e+05,35530081.0,23682.000000,71.4
613,Afghanistan,AFG,2016,549.582760,58.067580,84.137138,NaN,0.955493,8.841000,3.680295e+04,34656032.0,22770.000000,47.9
614,Afghanistan,AFG,2015,590.076474,58.067580,71.500000,NaN,0.993455,8.864000,2.424755e+05,33736494.0,23532.000000,36.1
615,Afghanistan,AFG,2014,625.339539,58.067580,89.500000,0.299445,1.298013,8.706000,1.078901e+05,32758020.0,25920.000000,36.1
616,Afghanistan,AFG,2013,647.966460,58.067580,67.259552,0.315602,1.076950,8.452000,1.094402e+07,31731688.0,21696.000000,35.6
617,Afghanistan,AFG,2012,648.511070,58.067580,69.100000,0.350371,1.175417,7.936000,1.073803e+07,30696958.0,17775.000000,35.8
618,Afghanistan,AFG,2011,599.297630,58.067580,43.222019,0.412017,1.821346,8.230000,4.708426e+07,29708599.0,25021.000000,35.8
619,Afghanistan,AFG,2010,550.514974,58.069111,42.700000,0.293837,1.945837,7.821000,2.757785e+07,28803167.0,21677.000000,35.8
620,Afghanistan,AFG,2009,444.184404,58.067580,44.854885,0.241723,2.087413,6.705000,9.476112e+04,28004331.0,NaN,35.8
